In [1]:
import nltk
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics
import os
from pathlib import Path
import re
import pickle
import random

NUM_JANELA=4

In [2]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

In [3]:
def getTiposEntidade():
    #return ['Problema','Teste','Tratamento','Anatomia']
    return ['ENT']

def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

def save_obj(name, obj):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    print('Load obj em: ', 'obj/' + name + '.pkl')
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [84]:
#pathTrain=r'../spanclassification/preProcessamento/data-ner-binario/nested_train.conll'
#pathDev=r'../spanclassification/preProcessamento/data-ner-binario/nested_dev.conll'
#pathTest=r'../spanclassification/preProcessamento/data-ner-binario/nested_test.conll'

pathTrain=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iobes/nested_ner_qa_train.conll'
pathDev=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iobes\nested_ner_qa_dev.conll'
pathTest=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iobes\nested_ner_qa_test.conll'

with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
#devdata[:2]
traindata[:2]

[[('Problema', '<sep>'),
  ('Dispneia', 'B-ENT'),
  ('importante', 'I-ENT'),
  ('aos', 'I-ENT'),
  ('esforços', 'E-ENT'),
  ('+', 'O'),
  ('dor', 'B-ENT'),
  ('tipo', 'I-ENT'),
  ('peso', 'I-ENT'),
  ('no', 'I-ENT'),
  ('peito', 'I-ENT'),
  ('no', 'I-ENT'),
  ('esforço', 'E-ENT'),
  ('.', 'O')],
 [('Teste', '<sep>'),
  ('Dispneia', 'O'),
  ('importante', 'O'),
  ('aos', 'O'),
  ('esforços', 'O'),
  ('+', 'O'),
  ('dor', 'O'),
  ('tipo', 'O'),
  ('peso', 'O'),
  ('no', 'O'),
  ('peito', 'O'),
  ('no', 'O'),
  ('esforço', 'O'),
  ('.', 'O')]]

In [85]:
traindata[0][0][0]

'Problema'

In [86]:
dicPostagger = load_obj('../spanclassification/obj/dic_postagger')
def tipoPostaggerTokens(token, dicPostagger):
    postagger = 'N' # na duvida é N
    if token.lower() in dicPostagger.keys():
        postagger = dicPostagger.get(token.lower())
    return postagger
tipoPostaggerTokens('coração', dicPostagger)

Load obj em:  obj/../spanclassification/obj/dic_postagger.pkl


'N'

In [87]:
tipos =['Problema','Teste','Tratamento','Anatomia']
tipos.index('Problema')

0

In [88]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster

def word2features(sent, i, tag):
    word = sent[i][0]
    postag = tipoPostaggerTokens(word, dicPostagger)
    cluster = word2cluster[word.lower()] if word.lower() in word2cluster else "0"
    features = {
        'bias': 1.0,
        'tag': tag,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'word.cluster': cluster
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-1:tag': tag,
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:word.cluster': cluster
        })
    else:
        features['BOS'] = True
    
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-2:tag': tag,
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:word.cluster': cluster
        })
    else:
        features['Second_word'] = True

    if i > 2:
        word1 = sent[i-3][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-3:tag': tag,
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
            '-3:word.cluster': cluster
        })
    else:
        features['Third_word'] = True
    
    if i > 3:
        word1 = sent[i-4][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-4:tag': tag,
            '-4:word.lower()': word1.lower(),
            '-4:word.istitle()': word1.istitle(),
            '-4:word.isupper()': word1.isupper(),
            '-4:postag': postag1,
            '-4:word.cluster': cluster
        })
    else:
        features['Fourth_word'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+1:tag': tag,
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:word.cluster': cluster
        })
    else:
        features['EOS'] = True
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+2:tag': tag,
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:word.cluster': cluster
        })
    else:
        features['Second_to_last'] = True
    if i < len(sent)-3:
        word1 = sent[i+3][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+3:tag': tag,
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
            '+3:word.cluster': cluster
        })
    else:
        features['Third_to_last'] = True
    
    if i < len(sent)-4:
        word1 = sent[i+4][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+4:tag': tag,
            '+4:word.lower()': word1.lower(),
            '+4:word.istitle()': word1.istitle(),
            '+4:word.isupper()': word1.isupper(),
            '+4:postag': postag1,
            '+4:word.cluster': cluster
        })
    else:
        features['Fourth_to_last'] = True
    

    return features


tipos =['Problema','Teste','Tratamento','Anatomia']
def sent2features(sent):
    tag = sent[0][0]
    #print(tag)
    #tag = tipos.index(tag)
    #print(tag)
    #if str(sent[i][0])!='nan':#esto es extra
    #if sent[i][1] == '<sep>':
    return [word2features(sent, i, tag) for i in range(len(sent)) if sent[i][1] != '<sep>']
       


#def sent2features(sent):
#    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    try:
        #return [label for token, label in sent]
        #tag = sent[0][0]
        #print(tag)
        #tag = tipos.index(tag
        #print(tag)
        #if str(sent[i][0])!='nan':#esto es extra
        #if sent[i][1] == '<sep>':
        return [label for token, label in sent if label != '<sep>']
    except:
        print(sent)
        raise
        

def sent2tokens(sent):
    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"clusters/cluster-50.tsv")
word2cluster = read_clusters(r"clusters/cluster-300.tsv")


In [96]:
sent2features(testdata[0])

[{'bias': 1.0,
  'tag': 'Problema',
  'word.lower()': 'lucas',
  'word[-3:]': 'cas',
  'word[:3]': 'Luc',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'N',
  'word.cluster': '0',
  '-1:tag': 'Problema',
  '-1:word.lower()': 'problema',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'N',
  '-1:word.cluster': '187',
  'Second_word': True,
  'Third_word': True,
  'Fourth_word': True,
  '+1:tag': 'Problema',
  '+1:word.lower()': ',',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'PU',
  '+1:word.cluster': '22',
  '+2:tag': 'Problema',
  '+2:word.lower()': '74',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False,
  '+2:postag': 'NUM',
  '+2:word.cluster': '299',
  '+3:tag': 'Problema',
  '+3:word.lower()': 'anos',
  '+3:word.istitle()': False,
  '+3:word.isupper()': False,
  '+3:postag': 'N',
  '+3:word.cluster': '134',
  '+4:tag': 'Problema',
  '+4:word.lower()': '.',
  '+4:word.i

In [90]:
#X_train = word2features(lista, word2cluster, dicPostagger)
#X_train = [sent2features(s, word2cluster) for s in train_sents]
X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

In [91]:
print(len(X_train[0]))

13


In [92]:
y_train[:10]

[['B-ENT',
  'I-ENT',
  'I-ENT',
  'E-ENT',
  'O',
  'B-ENT',
  'I-ENT',
  'I-ENT',
  'I-ENT',
  'I-ENT',
  'I-ENT',
  'E-ENT',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S-ENT', 'O', 'O', 'O'],
 ['S-ENT', 'O', 'S-ENT', 'O', 'S-ENT', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'S-ENT', 'O'],
 ['O', 'O', 'B-ENT', 'E-ENT', 'O', 'S-ENT', 'O', 'O']]

In [93]:
print(len(traindata))
print(len(X_train))
print(len(y_train))

5276
5276
5276


## Janela de 4 vizinhos antes e depois

In [94]:
import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 5276/5276 [00:02<00:00, 2308.86it/s]


loading dev data to CRFsuite: 100%|██████████| 1664/1664 [00:00<00:00, 2774.19it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 41074
Seconds required: 0.517

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.17  loss=24455.34 active=40997 precision=0.185  recall=0.200  F1=0.192  Acc(item/seq)=0.924 0.740  feature_norm=1.00
Iter 2   time=0.10  loss=22917.80 active=37940 precision=0.185  recall=0.200  F1=0.192  Acc(item/seq)=0.924 0.740  feature_norm=1.13
Iter 3   time=0.10  loss=21835.15 active=34699 precision=0.185  recall=0.200  F1=0.192  Acc(item/seq)=0.924 0.740  feature_norm=1.26
Iter 4   time=0.10  loss=20527.35 active=36404 precision=0.185  recall=0.200  F1=0.192  Acc(item/seq)=0.924 0.740  feature_norm=1.43
Iter 5   time=0.10  loss=18253.24 active=38392 pr

Iter 60  time=0.16  loss=5424.21  active=33535 precision=0.652  recall=0.386  F1=0.457  Acc(item/seq)=0.930 0.743  feature_norm=38.05
Iter 61  time=0.14  loss=5407.68  active=32934 precision=0.656  recall=0.380  F1=0.452  Acc(item/seq)=0.930 0.744  feature_norm=38.10
Iter 62  time=0.13  loss=5397.54  active=32123 precision=0.650  recall=0.396  F1=0.467  Acc(item/seq)=0.930 0.743  feature_norm=38.39
Iter 63  time=0.11  loss=5386.51  active=32019 precision=0.659  recall=0.387  F1=0.459  Acc(item/seq)=0.930 0.746  feature_norm=38.39
Iter 64  time=0.12  loss=5378.22  active=31533 precision=0.643  recall=0.390  F1=0.459  Acc(item/seq)=0.929 0.744  feature_norm=38.51
Iter 65  time=0.13  loss=5368.80  active=30831 precision=0.655  recall=0.383  F1=0.455  Acc(item/seq)=0.930 0.748  feature_norm=38.56
Iter 66  time=0.11  loss=5363.64  active=30416 precision=0.636  recall=0.396  F1=0.464  Acc(item/seq)=0.929 0.744  feature_norm=38.77
Iter 67  time=0.10  loss=5354.67  active=30449 precision=0.647

In [95]:
import eli5

eli5.show_weights(crf, top=30)

In [60]:
import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = True)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 5276/5276 [00:02<00:00, 2008.52it/s]


loading dev data to CRFsuite: 100%|██████████| 1664/1664 [00:00<00:00, 2143.24it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 37321
Seconds required: 0.537

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.14  loss=19818.86 active=37308 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=1.00
Iter 2   time=0.07  loss=19275.66 active=30946 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=0.97
Iter 3   time=0.07  loss=16841.95 active=24983 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=1.06
Iter 4   time=0.08  loss=15455.01 active=34984 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=1.30
Iter 5   time=0.08  loss=12929.47 active=35832 pr

Iter 60  time=0.10  loss=5583.41  active=26584 precision=0.656  recall=0.476  F1=0.524  Acc(item/seq)=0.925 0.730  feature_norm=39.34
Iter 61  time=0.11  loss=5579.26  active=26565 precision=0.670  recall=0.474  F1=0.525  Acc(item/seq)=0.927 0.734  feature_norm=39.39
Iter 62  time=0.12  loss=5574.90  active=26425 precision=0.646  recall=0.479  F1=0.525  Acc(item/seq)=0.924 0.730  feature_norm=39.50
Iter 63  time=0.10  loss=5572.94  active=26280 precision=0.671  recall=0.466  F1=0.517  Acc(item/seq)=0.927 0.735  feature_norm=39.63
Iter 64  time=0.11  loss=5567.02  active=26326 precision=0.644  recall=0.479  F1=0.525  Acc(item/seq)=0.924 0.730  feature_norm=39.78
Iter 65  time=0.11  loss=5563.38  active=26302 precision=0.662  recall=0.471  F1=0.520  Acc(item/seq)=0.926 0.733  feature_norm=39.86
Iter 66  time=0.11  loss=5560.17  active=26299 precision=0.639  recall=0.474  F1=0.519  Acc(item/seq)=0.924 0.730  feature_norm=39.93
Iter 67  time=0.09  loss=5556.92  active=26194 precision=0.658

In [59]:
pathTrain=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iob2/nested_ner_qa_train.conll'
pathDev=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iob2\nested_ner_qa_dev.conll'
pathTest=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iob2\nested_ner_qa_test.conll'

with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

import eli5

eli5.show_weights(crf, top=30)


loading training data to CRFsuite: 100%|██████████| 5276/5276 [00:02<00:00, 2468.49it/s]


loading dev data to CRFsuite: 100%|██████████| 1664/1664 [00:00<00:00, 2769.72it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 37318
Seconds required: 0.479

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.13  loss=19924.21 active=37305 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=1.00
Iter 2   time=0.07  loss=19404.27 active=30947 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=0.97
Iter 3   time=0.08  loss=17164.94 active=25189 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=1.04
Iter 4   time=0.07  loss=15876.13 active=35610 precision=0.308  recall=0.333  F1=0.320  Acc(item/seq)=0.924 0.740  feature_norm=1.30
Iter 5   time=0.07  loss=13968.04 active=36056 pr

Iter 60  time=0.07  loss=5741.25  active=28365 precision=0.693  recall=0.449  F1=0.500  Acc(item/seq)=0.928 0.739  feature_norm=36.29
Iter 61  time=0.07  loss=5734.37  active=28308 precision=0.675  recall=0.462  F1=0.512  Acc(item/seq)=0.927 0.737  feature_norm=36.52
Iter 62  time=0.08  loss=5727.38  active=28262 precision=0.691  recall=0.451  F1=0.502  Acc(item/seq)=0.928 0.739  feature_norm=36.67
Iter 63  time=0.07  loss=5720.57  active=28230 precision=0.680  recall=0.469  F1=0.520  Acc(item/seq)=0.927 0.737  feature_norm=36.90
Iter 64  time=0.08  loss=5714.45  active=28127 precision=0.684  recall=0.449  F1=0.499  Acc(item/seq)=0.927 0.738  feature_norm=37.04
Iter 65  time=0.08  loss=5708.86  active=28082 precision=0.674  recall=0.466  F1=0.517  Acc(item/seq)=0.927 0.736  feature_norm=37.22
Iter 66  time=0.07  loss=5703.99  active=28035 precision=0.675  recall=0.447  F1=0.496  Acc(item/seq)=0.926 0.738  feature_norm=37.34
Iter 67  time=0.07  loss=5699.28  active=27989 precision=0.666

# Flat

In [64]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster

def word2features(sent, i, tag):
    word = sent[i][0]
    postag = tipoPostaggerTokens(word, dicPostagger)
    cluster = word2cluster[word.lower()] if word.lower() in word2cluster else "0"
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'word.cluster': cluster
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:word.cluster': cluster
        })
    else:
        features['BOS'] = True
    
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:word.cluster': cluster
        })
    else:
        features['Second_word'] = True

    if i > 2:
        word1 = sent[i-1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
            '-3:word.cluster': cluster
        })
    else:
        features['Third_word'] = True
    
    if i > 3:
        word1 = sent[i-2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-4:word.lower()': word1.lower(),
            '-4:word.istitle()': word1.istitle(),
            '-4:word.isupper()': word1.isupper(),
            '-4:postag': postag1,
            '-4:word.cluster': cluster
        })
    else:
        features['Fourth_word'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:word.cluster': cluster
        })
    else:
        features['EOS'] = True
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:word.cluster': cluster
        })
    else:
        features['Second_to_last'] = True
    if i < len(sent)-3:
        word1 = sent[i+3][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
            '+3:word.cluster': cluster
        })
    else:
        features['Third_to_last'] = True
    
    if i < len(sent)-4:
        word1 = sent[i+4][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+4:word.lower()': word1.lower(),
            '+4:word.istitle()': word1.istitle(),
            '+4:word.isupper()': word1.isupper(),
            '+4:postag': postag1,
            '+4:word.cluster': cluster
        })
    else:
        features['Fourth_to_last'] = True
    

    return features


tipos =['Problema','Teste','Tratamento','Anatomia']
def sent2features(sent):
    tag = sent[0][0]
    #print(tag)
    #tag = tipos.index(tag)
    #print(tag)
    #if str(sent[i][0])!='nan':#esto es extra
    #if sent[i][1] == '<sep>':
    return [word2features(sent, i, tag) for i in range(len(sent)) if sent[i][1] != '<sep>']
       


#def sent2features(sent):
#    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    try:
        #return [label for token, label in sent]
        #tag = sent[0][0]
        #print(tag)
        #tag = tipos.index(tag
        #print(tag)
        #if str(sent[i][0])!='nan':#esto es extra
        #if sent[i][1] == '<sep>':
        return [label for token, label in sent if label != '<sep>']
    except:
        print(sent)
        raise
        

def sent2tokens(sent):
    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"clusters/cluster-50.tsv")
word2cluster = read_clusters(r"clusters/cluster-300.tsv")


In [65]:
pathTrain=r'../spanclassification/preProcessamento/data-ner-binario/nested_train.conll'
pathDev=r'../spanclassification/preProcessamento/data-ner-binario/nested_dev.conll'
pathTest=r'../spanclassification/preProcessamento/data-ner-binario/nested_test.conll'


with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

import eli5

eli5.show_weights(crf, top=30)


loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:00<00:00, 2134.64it/s]


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:00<00:00, 2193.59it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28168
Seconds required: 0.172

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.05  loss=8439.03  active=26304 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.50
Iter 2   time=0.02  loss=7591.92  active=27402 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.45
Iter 3   time=0.02  loss=6728.64  active=26977 precision=0.696  recall=0.576  F1=0.570  Acc(item/seq)=0.742 0.255  feature_norm=0.67
Iter 4   time=0.02  loss=5961.74  active=27538 precision=0.702  recall=0.700  F1=0.701  Acc(item/seq)=0.759 0.276  feature_norm=1.12
Iter 5   time=0.02  loss=4917.25  active=26906 pr

Iter 62  time=0.02  loss=751.03   active=11051 precision=0.903  recall=0.903  F1=0.903  Acc(item/seq)=0.921 0.680  feature_norm=42.42
Iter 63  time=0.02  loss=750.62   active=11009 precision=0.905  recall=0.903  F1=0.904  Acc(item/seq)=0.922 0.683  feature_norm=42.40
Iter 64  time=0.02  loss=750.24   active=10975 precision=0.903  recall=0.903  F1=0.903  Acc(item/seq)=0.921 0.680  feature_norm=42.40
Iter 65  time=0.02  loss=749.81   active=10936 precision=0.904  recall=0.903  F1=0.903  Acc(item/seq)=0.922 0.680  feature_norm=42.39
Iter 66  time=0.02  loss=749.39   active=10875 precision=0.903  recall=0.903  F1=0.903  Acc(item/seq)=0.922 0.678  feature_norm=42.40
Iter 67  time=0.02  loss=749.02   active=10856 precision=0.904  recall=0.902  F1=0.903  Acc(item/seq)=0.922 0.680  feature_norm=42.39
Iter 68  time=0.02  loss=748.72   active=10835 precision=0.903  recall=0.903  F1=0.903  Acc(item/seq)=0.922 0.678  feature_norm=42.40
Iter 69  time=0.02  loss=748.44   active=10832 precision=0.905

In [66]:
X_train[0]

[{'bias': 1.0,
  'word.lower()': 'dispneia',
  'word[-3:]': 'eia',
  'word[:3]': 'Dis',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'N',
  'word.cluster': '117',
  'BOS': True,
  'Second_word': True,
  'Third_word': True,
  'Fourth_word': True,
  '+1:word.lower()': 'importante',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'ADJ',
  '+1:word.cluster': '91',
  '+2:word.lower()': 'aos',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False,
  '+2:postag': 'ART',
  '+2:word.cluster': '187',
  '+3:word.lower()': 'esforços',
  '+3:word.istitle()': False,
  '+3:word.isupper()': False,
  '+3:postag': 'N',
  '+3:word.cluster': '116',
  '+4:word.lower()': '+',
  '+4:word.istitle()': False,
  '+4:word.isupper()': False,
  '+4:postag': 'N',
  '+4:word.cluster': '112'},
 {'bias': 1.0,
  'word.lower()': 'importante',
  'word[-3:]': 'nte',
  'word[:3]': 'imp',
  'word.isupper()': False,
  'word.istitle()': False,
  'wo

# sem cluster

In [106]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster

def word2features(sent, i, tag):
    word = sent[i][0]
    postag = tipoPostaggerTokens(word, dicPostagger)
    cluster = word2cluster[word.lower()] if word.lower() in word2cluster else "0"
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True
    
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
        })
    else:
        features['Second_word'] = True

    if i > 2:
        word1 = sent[i-3][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
        })
    else:
        features['Third_word'] = True
    
    if i > 3:
        word1 = sent[i-4][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-4:word.lower()': word1.lower(),
            '-4:word.istitle()': word1.istitle(),
            '-4:word.isupper()': word1.isupper(),
            '-4:postag': postag1,
        })
    else:
        features['Fourth_word'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
        })
    else:
        features['Second_to_last'] = True
    if i < len(sent)-3:
        word1 = sent[i+3][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
        })
    else:
        features['Third_to_last'] = True
    
    if i < len(sent)-4:
        word1 = sent[i+4][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+4:word.lower()': word1.lower(),
            '+4:word.istitle()': word1.istitle(),
            '+4:word.isupper()': word1.isupper(),
            '+4:postag': postag1,
        })
    else:
        features['Fourth_to_last'] = True
    

    return features


tipos =['Problema','Teste','Tratamento','Anatomia']
def sent2features(sent):
    tag = sent[0][0]
    #print(tag)
    #tag = tipos.index(tag)
    #print(tag)
    #if str(sent[i][0])!='nan':#esto es extra
    #if sent[i][1] == '<sep>':
    return [word2features(sent, i, tag) for i in range(len(sent)) if sent[i][1] != '<sep>']
       


#def sent2features(sent):
#    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    try:
        #return [label for token, label in sent]
        #tag = sent[0][0]
        #print(tag)
        #tag = tipos.index(tag
        #print(tag)
        #if str(sent[i][0])!='nan':#esto es extra
        #if sent[i][1] == '<sep>':
        return [label for token, label in sent if label != '<sep>']
    except:
        print(sent)
        raise
        

def sent2tokens(sent):
    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"clusters/cluster-50.tsv")
word2cluster = read_clusters(r"clusters/cluster-300.tsv")


In [107]:
pathTrain=r'../spanclassification/preProcessamento/data-ner-binario/nested_train.conll'
pathDev=r'../spanclassification/preProcessamento/data-ner-binario/nested_dev.conll'
pathTest=r'../spanclassification/preProcessamento/data-ner-binario/nested_test.conll'


with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

import eli5

eli5.show_weights(crf, top=30)


loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:00<00:00, 2794.48it/s]


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:00<00:00, 2718.95it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 24309
Seconds required: 0.161

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.04  loss=8515.54  active=22569 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.50
Iter 2   time=0.01  loss=7697.23  active=23797 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.43
Iter 3   time=0.01  loss=6935.67  active=23416 precision=0.653  recall=0.538  F1=0.509  Acc(item/seq)=0.725 0.252  feature_norm=0.63
Iter 4   time=0.01  loss=6198.78  active=23743 precision=0.668  recall=0.660  F1=0.664  Acc(item/seq)=0.734 0.243  feature_norm=1.06
Iter 5   time=0.01  loss=5268.41  active=23311 pr

Iter 61  time=0.02  loss=991.70   active=10588 precision=0.887  recall=0.876  F1=0.881  Acc(item/seq)=0.905 0.615  feature_norm=49.85
Iter 62  time=0.02  loss=991.35   active=10561 precision=0.885  recall=0.876  F1=0.881  Acc(item/seq)=0.904 0.613  feature_norm=49.88
Iter 63  time=0.02  loss=990.82   active=10538 precision=0.887  recall=0.876  F1=0.881  Acc(item/seq)=0.905 0.615  feature_norm=49.88
Iter 64  time=0.02  loss=990.48   active=10541 precision=0.885  recall=0.876  F1=0.880  Acc(item/seq)=0.904 0.611  feature_norm=49.90
Iter 65  time=0.02  loss=990.17   active=10517 precision=0.887  recall=0.876  F1=0.881  Acc(item/seq)=0.905 0.615  feature_norm=49.91
Iter 66  time=0.02  loss=989.82   active=10482 precision=0.883  recall=0.875  F1=0.879  Acc(item/seq)=0.903 0.611  feature_norm=49.93
Iter 67  time=0.02  loss=989.59   active=10474 precision=0.887  recall=0.876  F1=0.881  Acc(item/seq)=0.905 0.613  feature_norm=49.94
Iter 68  time=0.02  loss=989.24   active=10434 precision=0.883

In [111]:
y_pred = crf.predict(X_test)
y_pred[0:2]

[['O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ENT',
  'O',
  'O',
  'O',
  'ENT',
  'ENT',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [112]:
y_pred = crf.predict_marginals(X_test)
y_pred[0:2]

[[{'ENT': 0.0038813793913830288, 'O': 0.9961186206086164},
  {'ENT': 1.9944551205868242e-05, 'O': 0.999980055448794},
  {'ENT': 0.00011207313042365513, 'O': 0.9998879268695763},
  {'ENT': 0.004021347283993453, 'O': 0.9959786527160064},
  {'ENT': 1.5263619556929183e-05, 'O': 0.9999847363804429}],
 [{'ENT': 0.0044171949616992195, 'O': 0.9955828050383009},
  {'ENT': 0.1276924458342183, 'O': 0.8723075541657818},
  {'ENT': 0.23500850791729594, 'O': 0.7649914920827041},
  {'ENT': 0.6046463442999648, 'O': 0.3953536557000352},
  {'ENT': 0.023520161909973164, 'O': 0.976479838090027},
  {'ENT': 0.0004913433957638261, 'O': 0.9995086566042364},
  {'ENT': 9.261632392520856e-05, 'O': 0.9999073836760751},
  {'ENT': 0.00010634412531658378, 'O': 0.9998936558746839},
  {'ENT': 0.971449624106921, 'O': 0.028550375893079363},
  {'ENT': 1.881201973625513e-05, 'O': 0.9999811879802639},
  {'ENT': 0.006824927308790086, 'O': 0.99317507269121},
  {'ENT': 0.026967907501876283, 'O': 0.9730320924981238},
  {'ENT': 

# janela de 10 - nao melhora

In [103]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster

def word2features(sent, i, tag):
    word = sent[i][0]
    postag = tipoPostaggerTokens(word, dicPostagger)
    cluster = word2cluster[word.lower()] if word.lower() in word2cluster else "0"
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    
   
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
        })
    else:
        features['primeira_palavra'] = True
    
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
        })
    #else:
    #    features['segunda_palavra'] = True

    if i > 2:
        word1 = sent[i-3][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
        })
    #else:
    #    features['terceira_palavra'] = True
    
    if i > 3:
        word1 = sent[i-4][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-4:word.lower()': word1.lower(),
            '-4:word.istitle()': word1.istitle(),
            '-4:word.isupper()': word1.isupper(),
            '-4:postag': postag1,
        })
    #else:
    #    features['quarta_palavra'] = True
    
    if i > 4:
        word1 = sent[i-5][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-5:word.lower()': word1.lower(),
            '-5:word.istitle()': word1.istitle(),
            '-5:word.isupper()': word1.isupper(),
            '-5:postag': postag1,
        })
    #else:
    #    features['quinta_palavra'] = True
    

    if i > 5:
        word1 = sent[i-6][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-6:word.lower()': word1.lower(),
            '-6:word.istitle()': word1.istitle(),
            '-6:word.isupper()': word1.isupper(),
            '-6:postag': postag1,
        })
    #else:
    #    features['sexta_palavra'] = True
    

    if i > 6:
        word1 = sent[i-7][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-7:word.lower()': word1.lower(),
            '-7:word.istitle()': word1.istitle(),
            '-7:word.isupper()': word1.isupper(),
            '-7:postag': postag1,
        })
    #else:
    #    features['setima_palavra'] = True
    

    if i > 7:
        word1 = sent[i-8][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-8:word.lower()': word1.lower(),
            '-8:word.istitle()': word1.istitle(),
            '-8:word.isupper()': word1.isupper(),
            '-8:postag': postag1,
        })
    #else:
    #    features['oitava_palavra'] = True
    

    if i > 8:
        word1 = sent[i-9][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-9:word.lower()': word1.lower(),
            '-9:word.istitle()': word1.istitle(),
            '-9:word.isupper()': word1.isupper(),
            '-9:postag': postag1,
        })
    #else:
    #    features['nona_palavra'] = True
    

    if i > 9:
        word1 = sent[i-10][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-10:word.lower()': word1.lower(),
            '-10:word.istitle()': word1.istitle(),
            '-10:word.isupper()': word1.isupper(),
            '-10:postag': postag1,
        })
    #else:
    #    features['decima_palavra'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        })
    else:
        features['ultima_palavra'] = True
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
        })
    #else:
        #features['penultima_palavra'] = True
    if i < len(sent)-3:
        word1 = sent[i+3][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
        })
    #else:
    #    features['antepenultima_palavra'] = True
    
    if i < len(sent)-4:
        word1 = sent[i+4][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+4:word.lower()': word1.lower(),
            '+4:word.istitle()': word1.istitle(),
            '+4:word.isupper()': word1.isupper(),
            '+4:postag': postag1,
        })
    #else:
    #    features['Fourth_to_last'] = True
    
 
    if i < len(sent)-5:
        word1 = sent[i+5][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+5:word.lower()': word1.lower(),
            '+5:word.istitle()': word1.istitle(),
            '+5:word.isupper()': word1.isupper(),
            '+5:postag': postag1,
        })
    #else:
    #    features['Fourth_to_last'] = True
        
 
    if i < len(sent)-6:
        word1 = sent[i+6][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+6:word.lower()': word1.lower(),
            '+6:word.istitle()': word1.istitle(),
            '+6:word.isupper()': word1.isupper(),
            '+6:postag': postag1,
        })
    #else:
    #    features['Fourth_to_last'] = True        
        
 
    if i < len(sent)-7:
        word1 = sent[i+7][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+7:word.lower()': word1.lower(),
            '+7:word.istitle()': word1.istitle(),
            '+7:word.isupper()': word1.isupper(),
            '+7:postag': postag1,
        })
    #else:
    #    features['Fourth_to_last'] = True
        
 
    if i < len(sent)-8:
        word1 = sent[i+8][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+8:word.lower()': word1.lower(),
            '+8:word.istitle()': word1.istitle(),
            '+8:word.isupper()': word1.isupper(),
            '+8:postag': postag1,
        })
    #else:
    #    features['Fourth_to_last'] = True        
        
 
    if i < len(sent)-9:
        word1 = sent[i+9][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+9:word.lower()': word1.lower(),
            '+9:word.istitle()': word1.istitle(),
            '+9:word.isupper()': word1.isupper(),
            '+9:postag': postag1,
        })
    #else:
    #    features['Fourth_to_last'] = True
        
 
    if i < len(sent)-10:
        word1 = sent[i+10][0]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+10:word.lower()': word1.lower(),
            '+10:word.istitle()': word1.istitle(),
            '+10:word.isupper()': word1.isupper(),
            '+10:postag': postag1,
        })
    #else:
    #    features['Fourth_to_last'] = True        
    
    return features


tipos =['Problema','Teste','Tratamento','Anatomia']
def sent2features(sent):
    tag = sent[0][0]
    #print(tag)
    #tag = tipos.index(tag)
    #print(tag)
    #if str(sent[i][0])!='nan':#esto es extra
    #if sent[i][1] == '<sep>':
    return [word2features(sent, i, tag) for i in range(len(sent)) if sent[i][1] != '<sep>']
       


#def sent2features(sent):
#    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    try:
        #return [label for token, label in sent]
        #tag = sent[0][0]
        #print(tag)
        #tag = tipos.index(tag
        #print(tag)
        #if str(sent[i][0])!='nan':#esto es extra
        #if sent[i][1] == '<sep>':
        return [label for token, label in sent if label != '<sep>']
    except:
        print(sent)
        raise
        

def sent2tokens(sent):
    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"clusters/cluster-50.tsv")
word2cluster = read_clusters(r"clusters/cluster-300.tsv")


In [104]:
X_test = [sent2features(s) for s in testdata]
X_test[0]

[{'bias': 1.0,
  'word.lower()': 'lucas',
  'word[-3:]': 'cas',
  'word[:3]': 'Luc',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'N',
  'primeira_palavra': True,
  '+1:word.lower()': ',',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'PU',
  '+2:word.lower()': '74',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False,
  '+2:postag': 'NUM',
  '+3:word.lower()': 'anos',
  '+3:word.istitle()': False,
  '+3:word.isupper()': False,
  '+3:postag': 'N',
  '+4:word.lower()': '.',
  '+4:word.istitle()': False,
  '+4:word.isupper()': False,
  '+4:postag': 'PU'},
 {'bias': 1.0,
  'word.lower()': ',',
  'word[-3:]': ',',
  'word[:3]': ',',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'PU',
  '-1:word.lower()': 'lucas',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'N',
  '+1:word.lower()': '74',
  '+1:word.istitle()': False,
  '+1:word.isup

In [105]:
pathTrain=r'../spanclassification/preProcessamento/data-ner-binario/nested_train.conll'
pathDev=r'../spanclassification/preProcessamento/data-ner-binario/nested_dev.conll'
pathTest=r'../spanclassification/preProcessamento/data-ner-binario/nested_test.conll'


with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

import eli5

eli5.show_weights(crf, top=30)


loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:00<00:00, 1453.18it/s]


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:00<00:00, 1802.54it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 44334
Seconds required: 0.241

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.06  loss=8821.31  active=40320 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.50
Iter 2   time=0.02  loss=7844.16  active=43186 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.40
Iter 3   time=0.03  loss=7143.90  active=42123 precision=0.661  recall=0.530  F1=0.490  Acc(item/seq)=0.725 0.264  feature_norm=0.55
Iter 4   time=0.03  loss=6297.29  active=42950 precision=0.674  recall=0.659  F1=0.665  Acc(item/seq)=0.740 0.279  feature_norm=0.98
Iter 5   time=0.03  loss=5267.54  active=42626 pr

Iter 63  time=0.02  loss=890.11   active=14213 precision=0.891  recall=0.875  F1=0.882  Acc(item/seq)=0.906 0.615  feature_norm=45.87
Iter 64  time=0.03  loss=889.76   active=14114 precision=0.889  recall=0.875  F1=0.882  Acc(item/seq)=0.906 0.615  feature_norm=45.86
Iter 65  time=0.03  loss=889.10   active=14055 precision=0.889  recall=0.874  F1=0.881  Acc(item/seq)=0.905 0.615  feature_norm=45.85
Iter 66  time=0.02  loss=888.74   active=14020 precision=0.889  recall=0.876  F1=0.882  Acc(item/seq)=0.906 0.618  feature_norm=45.85
Iter 67  time=0.02  loss=888.33   active=13976 precision=0.891  recall=0.876  F1=0.883  Acc(item/seq)=0.907 0.618  feature_norm=45.84
Iter 68  time=0.03  loss=887.93   active=13933 precision=0.889  recall=0.875  F1=0.882  Acc(item/seq)=0.906 0.615  feature_norm=45.84
Iter 69  time=0.02  loss=887.61   active=13891 precision=0.890  recall=0.872  F1=0.880  Acc(item/seq)=0.905 0.615  feature_norm=45.84
Iter 70  time=0.02  loss=887.25   active=13894 precision=0.890

# FLAT

In [70]:
pathTrain=r'../spanclassification/preProcessamento/data-ner-binario/nested_train.conll'
pathDev=r'../spanclassification/preProcessamento/data-ner-binario/nested_dev.conll'
pathTest=r'../spanclassification/preProcessamento/data-ner-binario/nested_test.conll'


with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

import eli5

eli5.show_weights(crf, top=30)


loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:00<00:00, 3522.53it/s]


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:00<00:00, 3992.75it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 24302
Seconds required: 0.106

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.03  loss=8485.43  active=22634 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.50
Iter 2   time=0.01  loss=7672.41  active=23844 precision=0.359  recall=0.500  F1=0.418  Acc(item/seq)=0.717 0.214  feature_norm=0.44
Iter 3   time=0.01  loss=6886.65  active=23483 precision=0.653  recall=0.548  F1=0.528  Acc(item/seq)=0.727 0.245  feature_norm=0.65
Iter 4   time=0.01  loss=6162.55  active=23826 precision=0.670  recall=0.662  F1=0.666  Acc(item/seq)=0.735 0.243  feature_norm=1.08
Iter 5   time=0.02  loss=5230.35  active=23344 pr

Iter 64  time=0.01  loss=1011.07  active=10814 precision=0.886  recall=0.877  F1=0.881  Acc(item/seq)=0.905 0.625  feature_norm=50.07
Iter 65  time=0.01  loss=1010.64  active=10772 precision=0.886  recall=0.876  F1=0.881  Acc(item/seq)=0.905 0.625  feature_norm=50.08
Iter 66  time=0.01  loss=1010.34  active=10721 precision=0.886  recall=0.877  F1=0.881  Acc(item/seq)=0.905 0.627  feature_norm=50.10
Iter 67  time=0.01  loss=1009.96  active=10684 precision=0.887  recall=0.877  F1=0.881  Acc(item/seq)=0.905 0.627  feature_norm=50.09
Iter 68  time=0.01  loss=1009.69  active=10668 precision=0.886  recall=0.876  F1=0.881  Acc(item/seq)=0.904 0.627  feature_norm=50.10
Iter 69  time=0.01  loss=1009.40  active=10633 precision=0.886  recall=0.876  F1=0.881  Acc(item/seq)=0.904 0.625  feature_norm=50.11
Iter 70  time=0.01  loss=1009.14  active=10625 precision=0.886  recall=0.877  F1=0.881  Acc(item/seq)=0.905 0.625  feature_norm=50.12
Iter 71  time=0.01  loss=1008.83  active=10602 precision=0.885

In [53]:
#pathTrain=r'../spanclassification/preProcessamento/data-ner-binario/nested_train.conll'
#pathDev=r'../spanclassification/preProcessamento/data-ner-binario/nested_dev.conll'
#pathTest=r'../spanclassification/preProcessamento/data-ner-binario/nested_test.conll'

pathTrain=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iob2/nested_ner_qa_train.conll'
pathDev=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iob2\nested_ner_qa_dev.conll'
pathTest=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_iob2\nested_ner_qa_test.conll'

with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
#devdata[:2]
traindata[:2]

[[('Problema', '<sep>'),
  ('Dispneia', 'B-ENT'),
  ('importante', 'I-ENT'),
  ('aos', 'I-ENT'),
  ('esforços', 'I-ENT'),
  ('+', 'O'),
  ('dor', 'B-ENT'),
  ('tipo', 'I-ENT'),
  ('peso', 'I-ENT'),
  ('no', 'I-ENT'),
  ('peito', 'I-ENT'),
  ('no', 'I-ENT'),
  ('esforço', 'I-ENT'),
  ('.', 'O')],
 [('Teste', '<sep>'),
  ('Dispneia', 'O'),
  ('importante', 'O'),
  ('aos', 'O'),
  ('esforços', 'O'),
  ('+', 'O'),
  ('dor', 'O'),
  ('tipo', 'O'),
  ('peso', 'O'),
  ('no', 'O'),
  ('peito', 'O'),
  ('no', 'O'),
  ('esforço', 'O'),
  ('.', 'O')]]

In [54]:
#X_train = word2features(lista, word2cluster, dicPostagger)
#X_train = [sent2features(s, word2cluster) for s in train_sents]
X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 5276/5276 [00:01<00:00, 3214.99it/s]


loading dev data to CRFsuite: 100%|██████████| 1664/1664 [00:00<00:00, 3813.81it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28752
Seconds required: 0.322

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.10  loss=31946.61 active=24399 precision=0.209  recall=0.250  F1=0.228  Acc(item/seq)=0.837 0.000  feature_norm=1.00
Iter 2   time=0.06  loss=29556.86 active=26970 precision=0.209  recall=0.250  F1=0.228  Acc(item/seq)=0.837 0.000  feature_norm=1.04
Iter 3   time=0.06  loss=18788.19 active=20410 precision=0.479  recall=0.500  F1=0.489  Acc(item/seq)=0.930 0.740  feature_norm=2.21
Iter 4   time=0.06  loss=14470.98 active=27945 precision=0.729  recall=0.545  F1=0.566  Acc(item/seq)=0.936 0.740  feature_norm=2.65
Iter 5   time=0.06  loss=11821.99 active=27703 pr

Iter 66  time=0.07  loss=3114.03  active=10278 precision=0.957  recall=0.878  F1=0.908  Acc(item/seq)=0.977 0.814  feature_norm=36.38
Iter 67  time=0.07  loss=3112.81  active=10218 precision=0.955  recall=0.878  F1=0.908  Acc(item/seq)=0.977 0.812  feature_norm=36.43
Iter 68  time=0.07  loss=3111.71  active=10136 precision=0.957  recall=0.878  F1=0.909  Acc(item/seq)=0.978 0.813  feature_norm=36.40
Iter 69  time=0.07  loss=3110.49  active=10059 precision=0.955  recall=0.878  F1=0.908  Acc(item/seq)=0.977 0.812  feature_norm=36.43
Iter 70  time=0.07  loss=3109.47  active=10019 precision=0.957  recall=0.878  F1=0.908  Acc(item/seq)=0.978 0.813  feature_norm=36.42
Iter 71  time=0.07  loss=3108.45  active=9988  precision=0.956  recall=0.879  F1=0.909  Acc(item/seq)=0.978 0.813  feature_norm=36.45
Iter 72  time=0.08  loss=3107.56  active=9957  precision=0.957  recall=0.878  F1=0.908  Acc(item/seq)=0.978 0.813  feature_norm=36.45
Iter 73  time=0.08  loss=3106.65  active=9889  precision=0.956

In [55]:
#pathTrain=r'../spanclassification/preProcessamento/data-ner-binario/nested_train.conll'
#pathDev=r'../spanclassification/preProcessamento/data-ner-binario/nested_dev.conll'
#pathTest=r'../spanclassification/preProcessamento/data-ner-binario/nested_test.conll'

pathTrain=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_io/nested_ner_qa_train.conll'
pathDev=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_io\nested_ner_qa_dev.conll'
pathTest=r'C:\Users\lisat\OneDrive\jupyter notebook\bionested\data\data_ner_qa_io\nested_ner_qa_test.conll'

with open(pathTest, encoding='utf-8') as f:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathDev, encoding='utf-8') as f:
  devdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
with open(pathTrain, encoding='utf-8') as f:
  traindata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]
#devdata[:2]

traindata[:2]

#X_train = word2features(lista, word2cluster, dicPostagger)
#X_train = [sent2features(s, word2cluster) for s in train_sents]
X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 5276/5276 [00:01<00:00, 3617.95it/s]


loading dev data to CRFsuite: 100%|██████████| 1664/1664 [00:00<00:00, 3727.67it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26666
Seconds required: 0.323

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.08  loss=30120.40 active=26525 precision=0.279  recall=0.333  F1=0.304  Acc(item/seq)=0.837 0.000  feature_norm=1.00
Iter 2   time=0.05  loss=27076.68 active=21487 precision=0.279  recall=0.333  F1=0.304  Acc(item/seq)=0.837 0.000  feature_norm=0.95
Iter 3   time=0.05  loss=19720.94 active=19482 precision=0.967  recall=0.679  F1=0.677  Acc(item/seq)=0.933 0.740  feature_norm=1.27
Iter 4   time=0.05  loss=16178.99 active=25157 precision=0.966  recall=0.706  F1=0.724  Acc(item/seq)=0.938 0.740  feature_norm=1.68
Iter 5   time=0.05  loss=11317.65 active=25710 pr

Iter 64  time=0.05  loss=3752.88  active=12385 precision=0.953  recall=0.898  F1=0.922  Acc(item/seq)=0.973 0.795  feature_norm=43.39
Iter 65  time=0.05  loss=3751.29  active=12373 precision=0.955  recall=0.893  F1=0.920  Acc(item/seq)=0.972 0.795  feature_norm=43.46
Iter 66  time=0.05  loss=3750.16  active=12316 precision=0.953  recall=0.897  F1=0.921  Acc(item/seq)=0.973 0.794  feature_norm=43.49
Iter 67  time=0.05  loss=3749.10  active=12298 precision=0.955  recall=0.893  F1=0.920  Acc(item/seq)=0.972 0.796  feature_norm=43.51
Iter 68  time=0.05  loss=3747.64  active=12229 precision=0.953  recall=0.897  F1=0.921  Acc(item/seq)=0.973 0.794  feature_norm=43.54
Iter 69  time=0.05  loss=3747.12  active=12175 precision=0.956  recall=0.892  F1=0.919  Acc(item/seq)=0.972 0.796  feature_norm=43.57
Iter 70  time=0.05  loss=3745.37  active=12112 precision=0.952  recall=0.896  F1=0.921  Acc(item/seq)=0.972 0.792  feature_norm=43.60
Iter 71  time=0.05  loss=3744.57  active=12102 precision=0.955

In [103]:

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 5276/5276 [00:01<00:00, 3023.30it/s]


loading dev data to CRFsuite: 100%|██████████| 1664/1664 [00:00<00:00, 3398.79it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25273
Seconds required: 0.368

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.07  loss=17658.62 active=24523 precision=0.462  recall=0.500  F1=0.480  Acc(item/seq)=0.924 0.740  feature_norm=1.00
Iter 2   time=0.04  loss=15946.24 active=24813 precision=0.462  recall=0.500  F1=0.480  Acc(item/seq)=0.924 0.740  feature_norm=0.90
Iter 3   time=0.08  loss=12450.14 active=13498 precision=0.953  recall=0.520  F1=0.519  Acc(item/seq)=0.926 0.740  feature_norm=0.63
Iter 4   time=0.04  loss=11491.24 active=24241 precision=0.952  recall=0.533  F1=0.543  Acc(item/seq)=0.928 0.740  feature_norm=0.76
Iter 5   time=0.04  loss=9312.47  active=24497 pr

Iter 66  time=0.05  loss=3723.35  active=12549 precision=0.936  recall=0.844  F1=0.883  Acc(item/seq)=0.971 0.797  feature_norm=42.11
Iter 67  time=0.05  loss=3721.84  active=12509 precision=0.934  recall=0.846  F1=0.884  Acc(item/seq)=0.971 0.797  feature_norm=42.10
Iter 68  time=0.05  loss=3720.34  active=12406 precision=0.936  recall=0.845  F1=0.884  Acc(item/seq)=0.971 0.799  feature_norm=42.09
Iter 69  time=0.04  loss=3719.03  active=12331 precision=0.934  recall=0.847  F1=0.885  Acc(item/seq)=0.971 0.798  feature_norm=42.08
Iter 70  time=0.05  loss=3717.65  active=12304 precision=0.935  recall=0.844  F1=0.883  Acc(item/seq)=0.970 0.797  feature_norm=42.10
Iter 71  time=0.04  loss=3716.55  active=12244 precision=0.935  recall=0.846  F1=0.884  Acc(item/seq)=0.971 0.797  feature_norm=42.09
Iter 72  time=0.04  loss=3715.30  active=12195 precision=0.935  recall=0.844  F1=0.883  Acc(item/seq)=0.970 0.797  feature_norm=42.10
Iter 73  time=0.05  loss=3714.12  active=12125 precision=0.933

In [104]:
#print(X_test[0])
crf.predict(X_test)


[['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ENT',
  'ENT',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'ENT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  

In [106]:
# cluster 50

import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)

C:\Users\lisat\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  4.9min finished


              precision    recall  f1-score   support

         ENT      0.892     0.723     0.799      1806

   micro avg      0.892     0.723     0.799      1806
   macro avg      0.892     0.723     0.799      1806
weighted avg      0.892     0.723     0.799      1806



C:\Users\lisat\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['ENT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


NameError: name 'OUTPUT_PATH' is not defined

In [108]:
import eli5

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)

best params: {'c1': 0.05408072153584, 'c2': 0.025342765201054995}
best CV score: 0.7954955818528687
model size: 0.70M


C:\Users\lisat\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


In [110]:
# cluster 300

#X_train = word2features(lista, word2cluster, dicPostagger)
#X_train = [sent2features(s, word2cluster) for s in train_sents]
X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.1min finished
C:\Users\lisat\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['ENT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

         ENT      0.894     0.717     0.796      1806

   micro avg      0.894     0.717     0.796      1806
   macro avg      0.894     0.717     0.796      1806
weighted avg      0.894     0.717     0.796      1806

best params: {'c1': 0.22183219984727368, 'c2': 0.03087825495997237}
best CV score: 0.7995006033181377
model size: 0.47M


In [112]:
# cluster 300, janela 5

#X_train = word2features(lista, word2cluster, dicPostagger)
#X_train = [sent2features(s, word2cluster) for s in train_sents]
X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  5.9min finished


              precision    recall  f1-score   support

         ENT      0.892     0.709     0.790      1806

   micro avg      0.892     0.709     0.790      1806
   macro avg      0.892     0.709     0.790      1806
weighted avg      0.892     0.709     0.790      1806

best params: {'c1': 0.17041205830636758, 'c2': 0.00275779785894667}
best CV score: 0.8004865319740259
model size: 0.56M


In [121]:
tag_index = best_crf.classes_.index('ENT')
print(tag_index)
tag_index = best_crf.classes_.index('O')
print(tag_index)

0
1


In [125]:
import time

# class_weight={'0': 1, '1': 5})

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False,  class_weight={'0': 2, '1': 1})

tag_index = best_crf.classes_.index('ENT')
print('ENT:', tag_index)
tag_index = best_crf.classes_.index('O')
print('O', tag_index)

# sem balanceamento - 77,9 de F1
start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

#print('CRF model was trained!')

TypeError: __init__() got an unexpected keyword argument 'class_weight'

In [65]:
import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:00<00:00, 3501.49it/s]


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:00<00:00, 3423.07it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 26345
Seconds required: 0.100

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.03  loss=13044.14 active=26009 precision=0.143  recall=0.200  F1=0.167  Acc(item/seq)=0.717 0.214  feature_norm=1.00
Iter 2   time=0.02  loss=11803.30 active=24798 precision=0.143  recall=0.200  F1=0.167  Acc(item/seq)=0.717 0.214  feature_norm=1.03
Iter 3   time=0.02  loss=9435.91  active=24645 precision=0.483  recall=0.341  F1=0.312  Acc(item/seq)=0.769 0.243  feature_norm=1.88
Iter 4   time=0.02  loss=8234.84  active=25550 precision=0.501  recall=0.353  F1=0.364  Acc(item/seq)=0.796 0.250  feature_norm=2.11
Iter 5   time=0.02  loss=6902.72  active=25798 pr

Iter 64  time=0.02  loss=796.28   active=8051  precision=0.936  recall=0.844  F1=0.882  Acc(item/seq)=0.956 0.690  feature_norm=46.79
Iter 65  time=0.02  loss=795.95   active=8037  precision=0.933  recall=0.845  F1=0.881  Acc(item/seq)=0.956 0.685  feature_norm=46.81
Iter 66  time=0.02  loss=795.59   active=8027  precision=0.936  recall=0.844  F1=0.881  Acc(item/seq)=0.956 0.688  feature_norm=46.82
Iter 67  time=0.02  loss=795.35   active=8019  precision=0.933  recall=0.845  F1=0.881  Acc(item/seq)=0.956 0.685  feature_norm=46.84
Iter 68  time=0.02  loss=795.01   active=8005  precision=0.936  recall=0.844  F1=0.881  Acc(item/seq)=0.956 0.688  feature_norm=46.84
Iter 69  time=0.02  loss=794.77   active=7986  precision=0.933  recall=0.846  F1=0.881  Acc(item/seq)=0.956 0.688  feature_norm=46.87
Iter 70  time=0.02  loss=794.49   active=7971  precision=0.936  recall=0.844  F1=0.881  Acc(item/seq)=0.956 0.688  feature_norm=46.87
Iter 71  time=0.02  loss=794.26   active=7948  precision=0.933

## Janela de 3 vizinhos antes e depois


In [72]:
import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:00<00:00, 4303.40it/s]


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:00<00:00, 4037.73it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 21792
Seconds required: 0.097

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.03  loss=12913.99 active=21519 precision=0.143  recall=0.200  F1=0.167  Acc(item/seq)=0.717 0.214  feature_norm=1.00
Iter 2   time=0.02  loss=11685.19 active=20574 precision=0.143  recall=0.200  F1=0.167  Acc(item/seq)=0.717 0.214  feature_norm=1.09
Iter 3   time=0.02  loss=8670.78  active=20435 precision=0.473  recall=0.365  F1=0.337  Acc(item/seq)=0.775 0.248  feature_norm=2.50
Iter 4   time=0.02  loss=7264.99  active=21202 precision=0.503  recall=0.418  F1=0.441  Acc(item/seq)=0.825 0.255  feature_norm=2.82
Iter 5   time=0.02  loss=6352.30  active=21377 pr

Iter 63  time=0.02  loss=819.72   active=7182  precision=0.948  recall=0.845  F1=0.884  Acc(item/seq)=0.955 0.685  feature_norm=47.76
Iter 64  time=0.02  loss=819.39   active=7167  precision=0.947  recall=0.845  F1=0.883  Acc(item/seq)=0.955 0.685  feature_norm=47.77
Iter 65  time=0.02  loss=819.14   active=7146  precision=0.948  recall=0.845  F1=0.884  Acc(item/seq)=0.955 0.685  feature_norm=47.77
Iter 66  time=0.02  loss=818.86   active=7143  precision=0.948  recall=0.845  F1=0.884  Acc(item/seq)=0.955 0.685  feature_norm=47.78
Iter 67  time=0.02  loss=818.66   active=7132  precision=0.948  recall=0.844  F1=0.883  Acc(item/seq)=0.955 0.685  feature_norm=47.77
Iter 68  time=0.02  loss=818.39   active=7110  precision=0.948  recall=0.845  F1=0.884  Acc(item/seq)=0.955 0.685  feature_norm=47.79
Iter 69  time=0.02  loss=818.16   active=7101  precision=0.948  recall=0.843  F1=0.883  Acc(item/seq)=0.955 0.685  feature_norm=47.78
Iter 70  time=0.02  loss=817.92   active=7103  precision=0.948

## Janela de 2 vizinhos antes e depois

In [52]:
import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:00<00:00, 5221.01it/s]


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:00<00:00, 4914.06it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 17071
Seconds required: 0.085

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.02  loss=12838.87 active=16868 precision=0.143  recall=0.200  F1=0.167  Acc(item/seq)=0.717 0.214  feature_norm=1.00
Iter 2   time=0.01  loss=11581.15 active=16125 precision=0.143  recall=0.200  F1=0.167  Acc(item/seq)=0.717 0.214  feature_norm=1.12
Iter 3   time=0.01  loss=7980.63  active=16058 precision=0.484  recall=0.427  F1=0.426  Acc(item/seq)=0.814 0.267  feature_norm=2.84
Iter 4   time=0.01  loss=6654.99  active=16604 precision=0.522  recall=0.450  F1=0.476  Acc(item/seq)=0.846 0.308  feature_norm=3.28
Iter 5   time=0.01  loss=6065.41  active=16832 pr

Iter 73  time=0.01  loss=856.55   active=6080  precision=0.947  recall=0.849  F1=0.886  Acc(item/seq)=0.955 0.690  feature_norm=49.58
Iter 74  time=0.01  loss=856.40   active=6074  precision=0.948  recall=0.848  F1=0.886  Acc(item/seq)=0.955 0.692  feature_norm=49.59
Iter 75  time=0.01  loss=856.28   active=6062  precision=0.947  recall=0.850  F1=0.887  Acc(item/seq)=0.956 0.690  feature_norm=49.60
Iter 76  time=0.01  loss=856.15   active=6056  precision=0.948  recall=0.849  F1=0.887  Acc(item/seq)=0.956 0.692  feature_norm=49.61
Iter 77  time=0.01  loss=856.05   active=6054  precision=0.947  recall=0.850  F1=0.887  Acc(item/seq)=0.956 0.690  feature_norm=49.62
Iter 78  time=0.01  loss=855.93   active=6047  precision=0.948  recall=0.849  F1=0.886  Acc(item/seq)=0.955 0.692  feature_norm=49.63
Iter 79  time=0.02  loss=855.82   active=6049  precision=0.948  recall=0.849  F1=0.887  Acc(item/seq)=0.956 0.692  feature_norm=49.64
Iter 80  time=0.01  loss=855.71   active=6046  precision=0.949

In [54]:
#!pip install eli5

import eli5

eli5.show_weights(crf, top=30)

# Finding the optimal hyperparameters


## Cluster 50

In [73]:
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.6min finished


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000027F06C8FD00>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000027F06C8FCD0>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['Problema', 'Teste', 'Tratamento', 'Anatomia']),
                   verbose=1)

In [75]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.025025294252710672, 'c2': 0.0044731390067481855}
best CV score: 0.9085100773498992
model size: 0.38M


In [76]:
# com janela de 3
best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

              precision    recall  f1-score   support

    Problema      0.952     0.955     0.954       795
       Teste      0.930     0.909     0.920       308
  Tratamento      0.968     0.944     0.956       446
    Anatomia      0.895     0.716     0.795        95

   micro avg      0.950     0.929     0.939      1644
   macro avg      0.936     0.881     0.906      1644
weighted avg      0.949     0.929     0.939      1644



In [56]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.0710867769766405, 'c2': 0.007025286647796449}
best CV score: 0.9114276297168044
model size: 0.26M


In [57]:
# com janela de 2
best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

              precision    recall  f1-score   support

    Problema      0.955     0.955     0.955       795
       Teste      0.936     0.899     0.917       308
  Tratamento      0.968     0.935     0.951       446
    Anatomia      0.918     0.705     0.798        95

   micro avg      0.953     0.925     0.939      1644
   macro avg      0.944     0.874     0.905      1644
weighted avg      0.953     0.925     0.938      1644



C:\Users\lisat\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['Problema', 'Teste', 'Tratamento', 'Anatomia'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [58]:
import joblib
import os

OUTPUT_PATH = "models/"
OUTPUT_FILE = "best_crf_ner_cluster50"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(best_crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

['models/best_crf_ner_cluster50']

## Cluster 5


In [78]:
word2cluster = read_clusters(r"clusters/cluster-5.tsv")

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]


crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

OUTPUT_PATH = "models/"
OUTPUT_FILE = "best_crf_ner_cluster5"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(best_crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)



Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.3min finished


              precision    recall  f1-score   support

    Problema      0.948     0.948     0.948       795
       Teste      0.955     0.896     0.925       308
  Tratamento      0.961     0.948     0.955       446
    Anatomia      0.899     0.747     0.816        95

   micro avg      0.951     0.927     0.939      1644
   macro avg      0.941     0.885     0.911      1644
weighted avg      0.950     0.927     0.938      1644

best params: {'c1': 0.05058570873780858, 'c2': 0.02849278797510965}
best CV score: 0.9041782839454671
model size: 0.47M


## Cluster 10

In [79]:
word2cluster = read_clusters(r"clusters/cluster-10.tsv")

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]


crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

OUTPUT_PATH = "models/"
OUTPUT_FILE = "best_crf_ner_cluster10"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(best_crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)



Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.4min finished


              precision    recall  f1-score   support

    Problema      0.952     0.952     0.952       795
       Teste      0.933     0.903     0.917       308
  Tratamento      0.959     0.935     0.947       446
    Anatomia      0.895     0.716     0.795        95

   micro avg      0.948     0.925     0.936      1644
   macro avg      0.935     0.876     0.903      1644
weighted avg      0.947     0.925     0.935      1644

best params: {'c1': 0.03143530062660093, 'c2': 0.011407537452349636}
best CV score: 0.9025164743902709
model size: 0.44M


## Cluster 50

In [80]:
word2cluster = read_clusters(r"clusters/cluster-10.tsv")

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]


crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

OUTPUT_PATH = "models/"
OUTPUT_FILE = "best_crf_ner_cluster10"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(best_crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)



Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.5min finished


              precision    recall  f1-score   support

    Problema      0.957     0.951     0.954       795
       Teste      0.939     0.903     0.921       308
  Tratamento      0.961     0.935     0.948       446
    Anatomia      0.875     0.737     0.800        95

   micro avg      0.951     0.925     0.938      1644
   macro avg      0.933     0.881     0.906      1644
weighted avg      0.950     0.925     0.937      1644

best params: {'c1': 0.022118110448088233, 'c2': 0.027860023218946085}
best CV score: 0.9038591874502191
model size: 0.58M


## Cluster 100 

In [81]:
word2cluster = read_clusters(r"clusters/cluster-100.tsv")

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

OUTPUT_PATH = "models/"
OUTPUT_FILE = "best_crf_ner_cluster100"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(best_crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)



Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.4min finished


              precision    recall  f1-score   support

    Problema      0.945     0.956     0.951       795
       Teste      0.942     0.899     0.920       308
  Tratamento      0.970     0.937     0.953       446
    Anatomia      0.895     0.716     0.795        95

   micro avg      0.949     0.926     0.938      1644
   macro avg      0.938     0.877     0.905      1644
weighted avg      0.948     0.926     0.937      1644

best params: {'c1': 0.029081952698140505, 'c2': 0.01750489686490583}
best CV score: 0.911749142381645
model size: 0.45M


## Cluster 300 (melhor)

In [82]:
word2cluster = read_clusters(r"clusters/cluster-300.tsv")

X_train = [sent2features(s) for s in traindata]
y_train =[sent2labels(s) for s in traindata]
X_dev = [sent2features(s) for s in devdata]
y_dev = [sent2labels(s) for s in devdata]
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]


crf = crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=getTiposEntidade())

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

best_crf = rs.best_estimator_
y_pred = best_crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

import joblib
import os

OUTPUT_PATH = "models/"
OUTPUT_FILE = "best_crf_ner_cluster300"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(best_crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

eli5.show_weights(best_crf, top=30)



Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.4min finished


              precision    recall  f1-score   support

    Problema      0.958     0.952     0.955       795
       Teste      0.936     0.903     0.919       308
  Tratamento      0.977     0.960     0.968       446
    Anatomia      0.900     0.758     0.823        95

   micro avg      0.956     0.934     0.945      1644
   macro avg      0.943     0.893     0.916      1644
weighted avg      0.956     0.934     0.944      1644

best params: {'c1': 0.04358790768620843, 'c2': 0.01833886983260396}
best CV score: 0.9160357967942182
model size: 0.40M


## Agora com as regioes
(pode até ser binário)

In [83]:
# gerar as possibeis regioes, se são entidades, atribuir a label, senao, recebe 'O'

dic_sentencesTrain = load_obj(r'../spanclassification/obj/dic_sentencesTrain')
dic_sentencesDev = load_obj(r'../spanclassification/obj/dic_sentencesDev')
dic_sentencesTest = load_obj(r'../spanclassification/obj/dic_sentencesTest')
dic_sentencesTest[2]

Load obj em:  obj/../spanclassification/obj/dic_sentencesTrain.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesDev.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesTest.pkl


[[['Comorbidades', 0, 142],
  [':', 1, 154],
  ['DM', 2, 156],
  ['há', 3, 159],
  ['10', 4, 162],
  ['anos', 5, 165],
  ['em', 6, 170],
  ['uso', 7, 173],
  ['de', 8, 177],
  ['metformina', 9, 180],
  ['850mg', 10, 191],
  ['3', 11, 197],
  ['cp', 12, 199],
  ['/', 13, 201],
  ['dia', 14, 202],
  [',', 15, 205],
  ['acarbose', 16, 207],
  ['1', 17, 216],
  ['cp', 18, 218],
  ['/', 19, 220],
  ['dia', 20, 221],
  ['e', 21, 225],
  ['glicazida', 22, 227],
  ['60mg', 23, 237],
  ['2', 24, 242],
  ['cp', 25, 244],
  ['/', 26, 246],
  ['dia', 27, 247],
  ['e', 28, 251],
  ['insulina', 29, 253],
  ['(', 30, 262],
  ['24', 31, 263],
  ['-', 32, 266],
  ['0', 33, 268],
  ['-', 34, 270],
  ['24', 35, 272],
  [')', 36, 274],
  ['.', 37, 275]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [84]:
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))

[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


In [153]:
# aqui - gerar exemplos negativos tbm e add no dic
#dic_sentencesTest = load_obj(r'../spanclassification/obj/dic_sentencesTest')

def getListaCombinacaoEntidades(dic_sentences):
    listaRetorno=list()
    for key,value in dic_sentences.items():
        tokens=value[0]
        soTokens=[t[0] for t in tokens]
        soIndices=[t[1] for t in tokens]
        maiorIndice = soIndices[-1]
        #print('soIndices:', soIndices)
        entidadesPositivas=value[1]
        if len(entidadesPositivas)>0:
            #entidadesPositivasIndices=entidadesPositivas[1]
            entidadesPositivasIndices=[ent[1] for ent in entidadesPositivas]
        else:
            entidadesPositivasIndices=[]
        #print('tokens:', tokens)
        #print('entidadesPositivas:',entidadesPositivas)
        #print('entidadesPositivasIndices:',entidadesPositivasIndices)
        dicTokens={}
        for token in tokens:
            #print('token:', token)
            dicTokens[token[1]]=token[0]
        # fazendo a combinação
        indicesNovaEntidade=powerset(soIndices)
        #print(indicesNovaEntidade)
        listaEntidadesFrase=list()
        for indiceNovaEntidade in indicesNovaEntidade:
            vizinhosAntes = list()
            vizinhosDepois = list()
            label=0
            #print('indiceNovaEntidade:',indiceNovaEntidade)
            #print('entidadesPositivasIndices:',entidadesPositivasIndices)
            if indiceNovaEntidade in entidadesPositivasIndices:
                #print('ja tem')
                label=1
            tokensNovaEntidade = [dicTokens[i] for i in indiceNovaEntidade]
            inicioEnt = indiceNovaEntidade[0]
            fimEnt = indiceNovaEntidade[-1]
            #print('inicioEnt:', inicioEnt)
            #print('fimEnt:', fimEnt)
            for i in range(inicioEnt-1, inicioEnt-4, -1):
                if i>=0:
                    vizinhosAntes.append(i)
            for i in range(fimEnt+1, fimEnt+5, 1):
                if i<=maiorIndice:
                    vizinhosDepois.append(i)
            vizinhosAntes.sort()
            vizinhosAntesTokens=[dicTokens[i] for i in vizinhosAntes]
            vizinhosDepoisTokens=[dicTokens[i] for i in vizinhosDepois]
            listaEntidadesFrase.append([' '.join(tokensNovaEntidade), vizinhosAntesTokens, vizinhosDepoisTokens, label])
        listaRetorno.append(listaEntidadesFrase)
        listaEntidadesFrase=list()
        #listaEntidadesFrase.append([indiceNovaEntidade, vizinhosAntes, vizinhosDepois, label])
        #print(listaEntidadesFrase)
        #print(powerset(['dor', 'no', 'peito']))
    return listaRetorno

In [154]:
listaTest = getListaCombinacaoEntidades(dic_sentencesTest)
listaTest[0:2]

[[['Lucas', [], [',', '74', 'anos', '.'], 0],
  [',', ['Lucas'], ['74', 'anos', '.'], 0],
  ['74', ['Lucas', ','], ['anos', '.'], 0],
  ['anos', ['Lucas', ',', '74'], ['.'], 0],
  ['.', [',', '74', 'anos'], [], 0],
  ['Lucas ,', [], ['74', 'anos', '.'], 0],
  [', 74', ['Lucas'], ['anos', '.'], 0],
  ['74 anos', ['Lucas', ','], ['.'], 0],
  ['anos .', ['Lucas', ',', '74'], [], 0],
  ['Lucas , 74', [], ['anos', '.'], 0],
  [', 74 anos', ['Lucas'], ['.'], 0],
  ['74 anos .', ['Lucas', ','], [], 0],
  ['Lucas , 74 anos', [], ['.'], 0],
  [', 74 anos .', ['Lucas'], [], 0],
  ['Lucas , 74 anos .', [], [], 0]],
 [['Em', [], ['acompanhamento', 'no', 'ambualtorio', 'há'], 0],
  ['acompanhamento', ['Em'], ['no', 'ambualtorio', 'há', '5'], 0],
  ['no', ['Em', 'acompanhamento'], ['ambualtorio', 'há', '5', 'anos'], 0],
  ['ambualtorio',
   ['Em', 'acompanhamento', 'no'],
   ['há', '5', 'anos', 'por'],
   0],
  ['há',
   ['acompanhamento', 'no', 'ambualtorio'],
   ['5', 'anos', 'por', 'FA'],
   0],


In [161]:
listaDev = getListaCombinacaoEntidades(dic_sentencesDev)
listaDev[0:2]

[[['HAS', [], [',', 'ICC', ',', 'nega'], 1],
  [',', ['HAS'], ['ICC', ',', 'nega', 'DM'], 0],
  ['ICC', ['HAS', ','], [',', 'nega', 'DM', '.'], 1],
  [',', ['HAS', ',', 'ICC'], ['nega', 'DM', '.'], 0],
  ['nega', [',', 'ICC', ','], ['DM', '.'], 0],
  ['DM', ['ICC', ',', 'nega'], ['.'], 1],
  ['.', [',', 'nega', 'DM'], [], 0],
  ['HAS ,', [], ['ICC', ',', 'nega', 'DM'], 0],
  [', ICC', ['HAS'], [',', 'nega', 'DM', '.'], 0],
  ['ICC ,', ['HAS', ','], ['nega', 'DM', '.'], 0],
  [', nega', ['HAS', ',', 'ICC'], ['DM', '.'], 0],
  ['nega DM', [',', 'ICC', ','], ['.'], 0],
  ['DM .', ['ICC', ',', 'nega'], [], 0],
  ['HAS , ICC', [], [',', 'nega', 'DM', '.'], 0],
  [', ICC ,', ['HAS'], ['nega', 'DM', '.'], 0],
  ['ICC , nega', ['HAS', ','], ['DM', '.'], 0],
  [', nega DM', ['HAS', ',', 'ICC'], ['.'], 0],
  ['nega DM .', [',', 'ICC', ','], [], 0],
  ['HAS , ICC ,', [], ['nega', 'DM', '.'], 0],
  [', ICC , nega', ['HAS'], ['DM', '.'], 0],
  ['ICC , nega DM', ['HAS', ','], ['.'], 0],
  [', nega D

In [155]:
listaTrain = getListaCombinacaoEntidades(dic_sentencesTrain)
listaTrain[0:2]

[[['Dispneia', [], ['importante', 'aos', 'esforços', '+'], 0],
  ['importante', ['Dispneia'], ['aos', 'esforços', '+', 'dor'], 0],
  ['aos', ['Dispneia', 'importante'], ['esforços', '+', 'dor', 'tipo'], 0],
  ['esforços',
   ['Dispneia', 'importante', 'aos'],
   ['+', 'dor', 'tipo', 'peso'],
   0],
  ['+', ['importante', 'aos', 'esforços'], ['dor', 'tipo', 'peso', 'no'], 0],
  ['dor', ['aos', 'esforços', '+'], ['tipo', 'peso', 'no', 'peito'], 0],
  ['tipo', ['esforços', '+', 'dor'], ['peso', 'no', 'peito', 'no'], 0],
  ['peso', ['+', 'dor', 'tipo'], ['no', 'peito', 'no', 'esforço'], 0],
  ['no', ['dor', 'tipo', 'peso'], ['peito', 'no', 'esforço', '.'], 0],
  ['peito', ['tipo', 'peso', 'no'], ['no', 'esforço', '.'], 1],
  ['no', ['peso', 'no', 'peito'], ['esforço', '.'], 0],
  ['esforço', ['no', 'peito', 'no'], ['.'], 0],
  ['.', ['peito', 'no', 'esforço'], [], 0],
  ['Dispneia importante', [], ['aos', 'esforços', '+', 'dor'], 0],
  ['importante aos', ['Dispneia'], ['esforços', '+', 'do

In [190]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster


def word2features(sent, word2cluster, dicPostagger):
    try:
        features = list()
        entidades = sent[0]
        for entidade in entidades.split():
            postag = tipoPostaggerTokens(entidade, dicPostagger)
            features.extend([
            'bias',
            'word.lower=' + entidade.lower(),
            'word[-3:]=' + entidade[-3:],
            'word[:3]=' + entidade[:3],
            'word.isupper=%s' % entidade.isupper(),
            'word.istitle=%s' % entidade.istitle(),
            'word.isdigit=%s' % entidade.isdigit(),
            'word.cluster=%s' % word2cluster[entidade.lower()] if entidade.lower() in word2cluster else "0",
            'postag=' + postag,
            ])
        # palavras anteriores
        vizinhosAntes = sent[1] 
        if len(vizinhosAntes)>0:
            for num, vizinhoAntes in enumerate(vizinhosAntes):
                word1 = vizinhoAntes
                postag1 =  tipoPostaggerTokens(vizinhoAntes, dicPostagger)
                cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
                features.extend([
                    '-'+str(num+1)+':word.lower=' + word1.lower(),
                    '-'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '-'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '-'+str(num+1)+':postag=' + postag1,
                    '-'+str(num+1)+':word.cluster='+ cluster
                ])
        else:
            features.append('BOS')

        # próximas palavras
        vizinhosDepois = sent[2]
        if len(vizinhosDepois)>0:
            for num, vizinhoDepois in enumerate(vizinhosDepois):
                word1 = vizinhoDepois
                postag1 =  tipoPostaggerTokens(vizinhoDepois, dicPostagger)
                cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
                features.extend([
                    '+'+str(num+1)+':word.lower=' + word1.lower(),
                    '+'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '+'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '+'+str(num+1)+':postag=' + postag1,
                    '+'+str(num+1)+':word.cluster='+ cluster
                ])
        else:
            features.append('EOS')
    except:
        print('sent:', sent)
        raise
    return features


def sent2features(lista, word2cluster, dicPostagger):
    #return word2features(lista, word2cluster, dicPostagger)
    return [word2features(lista[i], word2cluster, dicPostagger) for i in range(len(lista))]

def sent2labels(lista):
    #return [label for _, _, _, label in lista]
    return [str(label) for _, _, _, label in lista]


#def sent2tokens(sent):
#    return [token for token, postag, label in sent]

word2cluster = read_clusters(r"clusters/cluster-50.tsv")
#word2cluster = read_clusters(r"clusters/cluster-5.tsv")
#word2cluster = read_clusters(r"clusters/cluster-10.tsv")
#word2cluster = read_clusters(r"clusters/cluster-100.tsv")
#word2cluster = read_clusters(r"clusters/cluster-300.tsv")

In [191]:
X_train = [sent2features(s, word2cluster, dicPostagger) for s in listaTrain]
y_train = [sent2labels(s) for s in listaTrain]
X_train[0]

[['bias',
  'word.lower=dispneia',
  'word[-3:]=eia',
  'word[:3]=Dis',
  'word.isupper=False',
  'word.istitle=True',
  'word.isdigit=False',
  'word.cluster=25',
  'postag=N',
  'BOS',
  '+1:word.lower=importante',
  '+1:word.istitle=False',
  '+1:word.isupper=False',
  '+1:postag=ADJ',
  '+1:word.cluster=26',
  '+2:word.lower=aos',
  '+2:word.istitle=False',
  '+2:word.isupper=False',
  '+2:postag=ART',
  '+2:word.cluster=43',
  '+3:word.lower=esforços',
  '+3:word.istitle=False',
  '+3:word.isupper=False',
  '+3:postag=N',
  '+3:word.cluster=25',
  '+4:word.lower=+',
  '+4:word.istitle=False',
  '+4:word.isupper=False',
  '+4:postag=N',
  '+4:word.cluster=17'],
 ['bias',
  'word.lower=importante',
  'word[-3:]=nte',
  'word[:3]=imp',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'word.cluster=26',
  'postag=ADJ',
  '-1:word.lower=dispneia',
  '-1:word.istitle=True',
  '-1:word.isupper=False',
  '-1:postag=N',
  '-1:word.cluster=25',
  '+1:word.lower=aos'

In [192]:
y_train[:2]

[['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['1',
  '0',
  '1',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [193]:
X_dev = [sent2features(s, word2cluster, dicPostagger) for s in listaDev]
y_dev = [sent2labels(s) for s in listaDev]

In [194]:
X_test = [sent2features(s, word2cluster, dicPostagger) for s in listaTest]
y_test = [sent2labels(s) for s in listaTest]

In [197]:
print(len(listaTrain))
print(len(X_train))
print(len(y_train))

1319
1319
1319


In [196]:
y_dev[0]

['1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [180]:
import time

#crfsuite is an implementation for Conditional Random Field

#!pip install sklearn-crfsuite
from sklearn_crfsuite import CRF

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         verbose='true',
         max_iterations = 100,
         all_possible_transitions = False)

start = time.time()
crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
#crf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {round(stop - start,2)}s")

print('CRF model was trained!')

loading training data to CRFsuite: 100%|██████████| 1319/1319 [00:38<00:00, 34.43it/s] 


loading dev data to CRFsuite: 100%|██████████| 416/416 [00:03<00:00, 106.28it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 24410
Seconds required: 4.494

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=1.06  loss=13787.03 active=24194 precision=0.491  recall=0.500  F1=0.495  Acc(item/seq)=0.981 0.214  feature_norm=1.00
Iter 2   time=0.56  loss=13752.73 active=23880 precision=0.491  recall=0.500  F1=0.495  Acc(item/seq)=0.981 0.214  feature_norm=1.00
Iter 3   time=1.03  loss=9775.09  active=13952 precision=0.491  recall=0.500  F1=0.495  Acc(item/seq)=0.981 0.214  feature_norm=0.64
Iter 4   time=0.54  loss=9286.92  active=23416 precision=0.491  recall=0.500  F1=0.495  Acc(item/seq)=0.981 0.214  feature_norm=0.60
Iter 5   time=0.56  loss=8512.58  active=23654 pr

Iter 61  time=0.74  loss=2651.51  active=14651 precision=0.865  recall=0.768  F1=0.809  Acc(item/seq)=0.988 0.498  feature_norm=61.28
Iter 62  time=0.96  loss=2646.23  active=14576 precision=0.863  recall=0.772  F1=0.811  Acc(item/seq)=0.988 0.495  feature_norm=61.43
Iter 63  time=0.86  loss=2641.57  active=14472 precision=0.867  recall=0.769  F1=0.810  Acc(item/seq)=0.988 0.500  feature_norm=61.55
Iter 64  time=0.86  loss=2637.81  active=14422 precision=0.863  recall=0.772  F1=0.811  Acc(item/seq)=0.988 0.493  feature_norm=61.74
Iter 65  time=0.64  loss=2634.25  active=14336 precision=0.866  recall=0.769  F1=0.810  Acc(item/seq)=0.988 0.495  feature_norm=61.83
Iter 66  time=0.63  loss=2631.00  active=14314 precision=0.864  recall=0.771  F1=0.810  Acc(item/seq)=0.988 0.493  feature_norm=61.95
Iter 67  time=0.65  loss=2627.14  active=14227 precision=0.864  recall=0.768  F1=0.809  Acc(item/seq)=0.988 0.488  feature_norm=62.06
Iter 68  time=0.66  loss=2623.74  active=14179 precision=0.860

In [181]:
import eli5

eli5.show_weights(crf, top=30)

In [ ]:
# janela de 4

In [ ]:
# janela de 2

In [ ]:
 # diferentes clusters